In [1]:
import os
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import cv2 

In [5]:
from keras.models import model_from_json

Using TensorFlow backend.


In [6]:
def read_model():
    model = model_from_json(open('architecture.json').read())
    model.load_weights('weights.h5')
    return model

In [7]:
model = read_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
face_cascade = cv2.CascadeClassifier('haar/haarcascade_frontalface_default.xml')

In [68]:
sunglasses = cv2.imread('images/sunglasses.png', cv2.IMREAD_UNCHANGED)

In [71]:
camera = cv2.VideoCapture(0)

In [72]:

#k = 1
while True:
#    k += 1
    
    (grabbed, frame) = camera.read()
    frame = cv2.flip(frame, 1)
    frame2 = np.copy(frame)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.25, 6)

    for (x, y, w, h) in faces:

        gray_face = gray[y : y+h, x : x+w]
        color_face = frame[y : y+h, x : x+w]

        gray_norm = gray_face / 255

        original_shape = gray_face.shape
        face_resized = cv2.resize(gray_norm, (96, 96), interpolation = cv2.INTER_AREA)
        face_resized_copy = face_resized.copy()
        face_resized = face_resized.reshape(1, 96, 96, 1)

        keypoints = model.predict(face_resized)
        #print(keypoints)

        keypoints = keypoints * 96

        face_resized_color = cv2.resize(color_face, (96, 96), interpolation = cv2.INTER_AREA)
        face_resized_color2 = np.copy(face_resized_color)

        points = keypoints[0]

        sunglass_width = int((points[0] - points[2]) * 1.9)
        sunglass_height = int((points[5] - points[1]) * 1.5)

        a = int(points[2]) - int(sunglass_height / 2)
        b = int(points[2]) + sunglass_height - int(sunglass_height / 2)
        c = int(points[2]) #+ int(sunglass_width / 5)
        d = int(points[2]) + sunglass_width #+ int(sunglass_width / 5)

        sunglass_resized = cv2.resize(sunglasses, (sunglass_width, sunglass_height), interpolation = cv2.INTER_CUBIC)
        transparent_region = sunglass_resized[:,:,:3] != 0

        face_resized_color[a : b, c : d, :][transparent_region] =  sunglass_resized[:,:,:3][transparent_region]

        frame[y:y+h, x:x+w] = cv2.resize(face_resized_color, original_shape, interpolation = cv2.INTER_CUBIC)

        for i in range(0, len(points), 2):
            cv2.circle(face_resized_color2, (points[i], points[i+1]), 1, (0, 255, 0), 1)

        frame2[y:y+h, x:x+w] = cv2.resize(face_resized_color2, original_shape, interpolation = cv2.INTER_CUBIC)
        
        cv2.imshow("Selfie filters", frame)
        cv2.imshow("keypoints", frame2)
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
camera.release()
cv2.destroyAllWindows()

#camera.release()

In [28]:
camera.release

<function VideoCapture.release>